In [306]:
import pandas as pd
import openpyxl
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors
import time
import datetime
from datetime import date

In [307]:
name_file_abiturs = 'data/abitur.xlsx'
name_file_person = 'data/person.xlsx'
path_to_end_folder_report = 'data'

In [308]:
df_abitur = pd.read_excel(name_file_abiturs,skiprows=3,usecols=['Абитуриент','Доп. статус','Состояние'])
df_person = pd.read_excel(name_file_person,sheet_name='Абитуриенты',skiprows=8,usecols=['ФИО','Нуждается в общежитии','Формирующее подр.','Направление, специальность, профессия','Сдан оригинал'])

In [309]:
wb = openpyxl.Workbook()
# Переименовываем лист
sheet = wb['Sheet']
sheet.title = 'Отчет'

In [310]:
df_person.head()

,ФИО,Нуждается в общежитии,Формирующее подр.,"Направление, специальность, профессия",Сдан оригинал
0,Бамбаров Арсалан Амгаланович,нет,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,нет
1,Галиагбаров Максим Романович,нет,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,нет
2,Литвинова Ульяна Алексеевна,нет,Отделение энергоснабжения,38.02.02 Страховое дело (по отраслям),нет


In [311]:
df_abitur.head()

,Абитуриент,Доп. статус,Состояние
0,Галиагбаров Максим Романович,NaN,Сданы ВИ
1,Бамбаров Арсалан Амгаланович,NaN,Сданы ВИ
2,Литвинова Ульяна Алексеевна,NaN,Сданы ВИ


In [312]:
df_person.shape

(3, 5)

In [313]:
df_abitur.shape

(3, 3)

In [314]:
# Соединяем оба датафрейма

In [315]:
cross_df = df_person.merge(df_abitur,how='inner',left_on='ФИО',right_on='Абитуриент')

In [316]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
cross_df['Нуждается в общежитии'] =cross_df['Нуждается в общежитии'].apply(lambda x:0 if x =='нет' else 1)
cross_df['Сдан оригинал'] =cross_df['Сдан оригинал'].apply(lambda x:0 if x =='нет' else 1)
cross_df['Состояние'] =cross_df['Состояние'].apply(lambda x:1 if x =='Забрал документы' else 0)


In [317]:
# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
cross_df['Доп. статус'].fillna('',inplace=True)
cross_df['Доп. статус'] = cross_df['Доп. статус'].apply(lambda x:1 if 'Сирота;' in x else 0)

In [318]:
cross_df['for_counting'] = 1

In [319]:
svod_df = pd.DataFrame.pivot_table(cross_df,index=['Формирующее подр.','Направление, специальность, профессия'],
                     values=['for_counting','Состояние','Сдан оригинал','Доп. статус','Нуждается в общежитии'],
                     aggfunc='sum')

In [320]:
svod_df.head()

,,for_counting,Доп. статус,Нуждается в общежитии,Сдан оригинал,Состояние
Формирующее подр.,"Направление, специальность, профессия",,,,,
Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,2,0,0,0,0
Отделение энергоснабжения,38.02.02 Страховое дело (по отраслям),1,0,0,0,0


In [321]:
svod_df.columns = ['Сдали всего','Сирот чел.','Нуждается в общежитии чел.','Сдано оригиналов','Забрали заявления чел.']

In [322]:
svod_df['Итого'] = svod_df['Сдали всего'] - svod_df['Забрали заявления чел.']

In [323]:
# Меняем местами столбцы
out_df = svod_df.reindex(columns=['Сдали всего','Забрали заявления чел.','Итого','Сдано оригиналов','Сирот чел.','Нуждается в общежитии чел.'])

In [324]:
for r in dataframe_to_rows(out_df,index =True,header=True):
    if len(r) != 1:
        wb['Отчет'].append(r)

In [325]:
# # Настраиваем выходной файл
wb['Отчет'].column_dimensions['A'].width =30
wb['Отчет'].column_dimensions['B'].width =50
wb['Отчет']['B2'].alignment = Alignment(wrap_text=True)
wb['Отчет'].column_dimensions['C'].width =20
wb['Отчет'].column_dimensions['D'].width =20
wb['Отчет'].column_dimensions['F'].width =20
wb['Отчет'].column_dimensions['G'].width =20
wb['Отчет'].column_dimensions['H'].width =30
wb['Отчет']['H1'].alignment = Alignment(wrap_text=True)

In [326]:
 # Получаем текущее время для того чтобы использовать в названии
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_report}/Ежедневный отчет приемной комиссии ГБПОУ БРИТ {current_time}.xlsx')

In [327]:
out_df

,,Сдали всего,Забрали заявления чел.,Итого,Сдано оригиналов,Сирот чел.,Нуждается в общежитии чел.
Формирующее подр.,"Направление, специальность, профессия",,,,,,
Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,2,0,2,0,0,0
Отделение энергоснабжения,38.02.02 Страховое дело (по отраслям),1,0,1,0,0,0


In [354]:
res_df = out_df.reset_index()

In [355]:
# Суммируем и Разворачиваем столбец в строку
sum_row=res_df.sum(axis=0).to_frame().transpose()


In [358]:
all_out_df = pd.concat([res_df,sum_row],axis=0)

In [359]:
all_out_df

,Формирующее подр.,"Направление, специальность, профессия",Сдали всего,Забрали заявления чел.,Итого,Сдано оригиналов,Сирот чел.,Нуждается в общежитии чел.
0,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,2,0,2,0,0,0
1,Отделение энергоснабжения,38.02.02 Страховое дело (по отраслям),1,0,1,0,0,0
0,Отделение железнодорожного транспортаОтделение...,23.01.09 Машинист локомотива38.02.02 Страховое...,3,0,3,0,0,0
